In [5]:
import recommendations as rec
import similarity as sim
import cProfile
from numba import jit

In [6]:
! python -m cProfile -s cumtime exec_2.py

#### logi testów

-  Wywołanie pliku exec_2.py z poniższą linijką kodu trwało ponad 413 sekund:

rec.executeDictSimMovies(prefs_file='datasets/prefs.json', target_file='datasets/similar_movies_2019.json', quantile=0.998, distance=sim.pearson)

         1260151978 function calls (1260104488 primitive calls) in 413.243 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    415/1    0.011    0.000  413.243  413.243 {built-in method builtins.exec}
        1    0.192    0.192  413.243  413.243 exec_2.py:4(<module>)
        1    0.676    0.676  412.630  412.630 recommendations.py:178(executeDictSimMovies)
        1    0.002    0.002  389.815  389.815 recommendations.py:96(buildRecomSet)
       91    0.001    0.000  389.807    4.284 recommendations.py:10(topMatches)
       91    1.908    0.021  389.804    4.284 recommendations.py:11(<listcomp>)
     8190   66.046    0.008  387.896    0.047 similarity.py:21(pearson)
    40951   58.420    0.001  302.205    0.007 {built-in method builtins.sum}
157287324   43.642    0.000   70.581    0.000 similarity.py:48(<genexpr>)
157287324   43.346    0.000   70.357    0.000 similarity.py:47(<genexpr>)
314574648   53.954    0.000   53.954    0.000 {built-in method math.pow}
157287324   47.637    0.000   47.637    0.000 similarity.py:43(<genexpr>)
157287324   27.893    0.000   27.893    0.000 similarity.py:45(<genexpr>)
157287324   27.317    0.000   27.317    0.000 similarity.py:44(<genexpr>)

- Widząc, że najwięcej czasu pochłania wykonywanie funkcji z pliku similarity.py, przepiąłem się z samodzielnie napisanej korelacji Pearsona na wersję z biblioteki numpy. Czas zaledwie ponad 168 sekund!:

rec.executeDictSimMovies(prefs_file='datasets/prefs.json', target_file='datasets/similar_movies_2019.json', quantile=0.998, distance=sim.pearson_numpy)

159623920 function calls (159576430 primitive calls) in 168.622 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    415/1    0.011    0.000  168.622  168.622 {built-in method builtins.exec}
        1    0.166    0.166  168.622  168.622 exec_2.py:4(<module>)
        1    0.675    0.675  168.037  168.037 recommendations.py:178(executeDictSimMovies)
        1    0.002    0.002  145.382  145.382 recommendations.py:96(buildRecomSet)
       91    0.001    0.000  145.375    1.598 recommendations.py:10(topMatches)
       91    3.743    0.041  145.370    1.597 recommendations.py:11(<listcomp>)
     8190   66.243    0.008  141.627    0.017 similarity.py:57(pearson_numpy)
     8190   23.179    0.003   23.179    0.003 similarity.py:63(<listcomp>)
        1    0.000    0.000   21.395   21.395 dataset.py:68(openJson)
        1    0.039    0.039   21.395   21.395 __init__.py:274(load)
     8190   20.877    0.003   20.877    0.003 similarity.py:64(<listcomp>)
157279179   19.618    0.000   19.618    0.000 {method 'setdefault' of 'dict' objects}
        3    0.000    0.000   16.851    5.617 __init__.py:302(loads)
        3    0.000    0.000   16.851    5.617 decoder.py:334(decode)
        3   16.851    5.617   16.851    5.617 decoder.py:345(raw_decode)
     8190    0.205    0.000   11.603    0.001 function_base.py:3097(corrcoef)
     8190    0.438    0.000   11.250    0.001 function_base.py:2888(cov)
    90181    8.122    0.000    8.122    0.000 {built-in method numpy.core.multiarray.array}
    16384    0.015    0.000    7.835    0.000 numeric.py:463(asarray)

- Ponad 66 sekund spośród 168 sekund w totalu przypada na funkcję pearson_numpy (w której wywołuję pearsona z biblioteki numpy). W ramach tej funkcji aż ponad 44 sekundy to same operacje typu lista składana (listcomp).

- zmieniam poniższą funkcję

In [8]:
def pearson_numpy(prefs, pos1, pos2):
    inter = {}
    for item in prefs[pos1]:
        if item in prefs[pos2]:
            inter.setdefault(item)

    x = [prefs[pos1][item] for item in inter]
    y = [prefs[pos2][item] for item in inter]

    r = np.corrcoef(x=x, y=y)

    return round(r[0][1], 6)

na:

In [ ]:
def pearson_numpy(prefs, pos1, pos2):

    inter = [item for item in prefs[pos1] if item in prefs[pos2]]
    # inter = prefs[pos1].keys() & prefs[pos2].keys()

    x = [prefs[pos1][item] for item in inter]
    y = [prefs[pos2][item] for item in inter]

    r = np.corrcoef(x=x, y=y)

    return round(r[0][1], 6)

Dzięki temu czas spadł z 168-170 sekund do 133-134 sekund. Dla oszczędności czasu obcinam parametr percentile do wartości 0.999. Wynik to 60-61 sekund.


rec.executeDictSimMovies(prefs_file='datasets/prefs.json', target_file='datasets/similar_movies_2019_nd_array.json', percentile=0.999, distance=sim.pearson_numpy)


1644987 function calls (1628097 primitive calls) in 60.451 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    415/1    0.011    0.000   60.451   60.451 {built-in method builtins.exec}
        1    0.129    0.129   60.451   60.451 exec_2.py:4(<module>)
        1    0.730    0.730   59.918   59.918 recommendations.py:178(executeDictSimMovies)
        1    0.001    0.001   37.107   37.107 recommendations.py:96(buildRecomSet)
       46    0.000    0.000   37.103    0.807 recommendations.py:10(topMatches)
       46    1.187    0.026   37.102    0.807 recommendations.py:11(<listcomp>)
     2070    0.042    0.000   35.915    0.017 similarity.py:57(pearson_numpy)
        1    0.000    0.000   21.577   21.577 dataset.py:68(openJson)
        1    0.050    0.050   21.577   21.577 __init__.py:274(load)
        3    0.000    0.000   16.999    5.666 __init__.py:302(loads)
        3    0.000    0.000   16.999    5.666 decoder.py:334(decode)
        3   16.999    5.666   16.999    5.666 decoder.py:345(raw_decode)
     2070   16.519    0.008   16.519    0.008 similarity.py:59(<listcomp>)
     2070    7.832    0.004    7.832    0.004 similarity.py:62(<listcomp>)
     2070    7.641    0.004    7.641    0.004 similarity.py:63(<listcomp>)
        1    1.764    1.764    4.528    4.528 {method 'read' of '_io.TextIOWrapper' objects}
     2070    0.106    0.000    3.852    0.002 function_base.py:3097(corrcoef)
     2070    0.130    0.000    3.705    0.002 function_base.py:2888(cov)
    22861    2.974    0.000    2.974    0.000 {built-in method numpy.core.multiarray.array}
     4144    0.005    0.000    2.865    0.001 numeric.py:463(asarray)
        7    0.000    0.000    2.765    0.395 cp1252.py:22(decode)
        7    2.765    0.395    2.765    0.395 {built-in method _codecs.charmap_decode}

In [15]:
i = 0
for i in range(1000):
    if i % 20 == 0:
        print(i)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
